In [1]:
# load packages
import os
import sys
import re
import pandas as pd
import numpy as np
import gensim
import nltk

In [2]:
# load data
data = pd.read_csv("data/news-headlines.csv")

In [3]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
